In [143]:
from io import StringIO
import pandas as pd
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}

In [144]:
# LEER FALLECIDOS POR COVID
url = 'https://files.minsa.gob.pe/s/t9AFqRbXw3F55Ho/download'
s = requests.get(url, headers= headers).text
df_fall = pd.read_csv(StringIO(s), sep=";", encoding='utf_8')

C:\Users\Miguel\AppData\Local\Temp\ipykernel_16612\4112748432.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fall = pd.read_csv(StringIO(s), sep=";", encoding='utf_8')


In [147]:
# LEER POSITIVOS POR COVID
url = 'https://files.minsa.gob.pe/s/eRqxR35ZCxrzNgr/download'
s = requests.get(url, headers= headers).text
df_posi = pd.read_csv(StringIO(s), sep=";", encoding='utf_8')

In [145]:
# LEER COVID MUNDIAL
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
s = requests.get(url, headers= headers).text
df_mundo = pd.read_csv(StringIO(s), sep=",", encoding='utf_8')

C:\Users\Miguel\AppData\Local\Temp\ipykernel_16612\3958896470.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mundo = pd.read_csv(StringIO(s), sep=",", encoding='utf_8')


In [146]:
# LEER DEPARTAMENTOS
df_dep = pd.read_excel("departamentos.xlsx")

In [148]:
# CAMBIANDO NOMBRE DE COLUMNAS A MINUSCULAS
df_fal = df_fall.rename(str.lower, axis='columns')
df_pos = df_posi.rename(str.lower, axis='columns')
df_dep = df_dep.rename(str.lower, axis='columns')
df_mun = df_mundo.rename(str.lower, axis='columns')

In [152]:
# -----------------------------------------------------------------
# ETL DE FALLECIDOS POR COVID
# -----------------------------------------------------------------

# CAMBIANDO EL CAMPO FECHA A TIPO STRING, DARLE FORMATO AAAA-MM-01
df_fal['fecha_fallecimiento'] = df_fal['fecha_fallecimiento'].astype('string').str[:8]
df_fal['fecha_fallecimiento'] = df_fal['fecha_fallecimiento'].str[:4] + '-' + df_fal['fecha_fallecimiento'].str[4:6] + '-01'

In [154]:
df_fal.head()

,fecha_corte,fecha_fallecimiento,edad_declarada,sexo,clasificacion_def,departamento,provincia,distrito,ubigeo,uuid,etapa_de_vida,pais_x,paiscode_x,departamentocode_x,pais_y,paiscode_y,departamentocode_y
0,20230517,2021--0-01,88,FEMENINO,Criterio SINADEF,AREQUIPA,AREQUIPA,PAUCARPATA,040112,13889730.0,MAYOR,PERU,PE,PE-ARE,PERU,PE,PE-ARE
1,20230517,2021--0-01,58,MASCULINO,Criterio SINADEF,AREQUIPA,AREQUIPA,PAUCARPATA,040112,13889749.0,ADULTO,PERU,PE,PE-ARE,PERU,PE,PE-ARE
2,20230517,2021--0-01,84,MASCULINO,Criterio serolÃ³gico,LA LIBERTAD,SANCHEZ CARRION,CHUGAY,130902,13889932.0,MAYOR,PERU,PE,PE-LAL,PERU,PE,PE-LAL
3,20230517,2021--0-01,64,MASCULINO,Criterio virolÃ³gico,LA LIBERTAD,SANTIAGO DE CHUCO,CACHICADAN,131003,13889956.0,MAYOR,PERU,PE,PE-LAL,PERU,PE,PE-LAL
4,20230517,2021--0-01,62,MASCULINO,Criterio SINADEF,LIMA,HUARAL,CHANCAY,150605,13890034.0,MAYOR,PERU,PE,PE-LIM,PERU,PE,PE-LIM


In [155]:
# HALLANDO 'etapa_de_vida'
df_fal['etapa_de_vida']=df_fal['edad_declarada'].apply(lambda x: 'NIÑO' if x<=9 else ('ADOLESCENTE' if x<=19 else ('JOVEN' if x<=29 else ('ADULTO' if x<=59 else 'MAYOR'))))

# JOIN ENTRE DATAFRAME FALLECIDOS Y DEPARTAMENTOS
df_fal = df_fal.merge(df_dep, left_on='departamento', right_on='departamento', how='left')

# CREANDO DATAFRAME FALLECIDOS POR SEXO Y ETAPA DE VIDA Y POR DIA
df_fallecido = df_fal.groupby(['departamentocode', 'fecha_fallecimiento', 'sexo', 'etapa_de_vida'])['departamentocode'].count().reset_index(name='cantidad')
df_fallecido = df_fallecido.rename(columns={'fecha_fallecimiento': 'fecha'})

In [156]:
df_fallecido.head()

,departamentocode,fecha,sexo,etapa_de_vida,cantidad
0,PE-AMA,2020--0-01,FEMENINO,ADOLESCENTE,3
1,PE-AMA,2020--0-01,FEMENINO,ADULTO,54
2,PE-AMA,2020--0-01,FEMENINO,JOVEN,9
3,PE-AMA,2020--0-01,FEMENINO,MAYOR,95
4,PE-AMA,2020--0-01,FEMENINO,NIÑO,3


In [157]:
# -----------------------------------------------------------------
# ETL DE POSITIVOS POR COVID
# -----------------------------------------------------------------

# CAMBIANDO EL CAMPO FECHA A TIPO STRING, DARLE FORMATO AAAA-MM-01
df_pos['fecha_resultado'] = df_pos['fecha_resultado'].astype('string').str[:8]
df_pos['fecha_resultado'] = df_pos['fecha_resultado'].str[:4] + '-' + df_pos['fecha_resultado'].str[4:6] + '-01'

In [160]:
df_pos.head()

,fecha_corte,departamento,provincia,distrito,metododx,edad,sexo,fecha_resultado,ubigeo,id_persona,etapa_de_vida,pais_x,paiscode_x,departamentocode_x,pais_y,paiscode_y,departamentocode_y
0,20230517,AREQUIPA,AREQUIPA,YANAHUARA,PCR,78.0,FEMENINO,2022-01-01,40126.0,13889713.0,MAYOR,PERU,PE,PE-ARE,PERU,PE,PE-ARE
1,20230517,AREQUIPA,AREQUIPA,PAUCARPATA,AG,67.0,MASCULINO,2021-06-01,40112.0,13889735.0,MAYOR,PERU,PE,PE-ARE,PERU,PE,PE-ARE
2,20230517,AREQUIPA,AREQUIPA,PAUCARPATA,PR,72.0,FEMENINO,2020-07-01,40112.0,13889743.0,MAYOR,PERU,PE,PE-ARE,PERU,PE,PE-ARE
3,20230517,LIMA,LIMA,LIMA,PR,23.0,MASCULINO,2021-01-01,150101.0,13889774.0,JOVEN,PERU,PE,PE-LIM,PERU,PE,PE-LIM
4,20230517,JUNIN,HUANCAYO,HUANCAYO,AG,24.0,FEMENINO,2022-01-01,120101.0,13889793.0,JOVEN,PERU,PE,PE-JUN,PERU,PE,PE-JUN


In [161]:

# HALLANDO 'etapa_de_vida'
df_pos['etapa_de_vida']=df_pos['edad'].apply(lambda x: 'NIÑO' if x<=9 else ('ADOLESCENTE' if x<=19 else ('JOVEN' if x<=29 else ('ADULTO' if x<=59 else 'MAYOR'))))

# JOIN ENTRE DATAFRAME POSITIVOS Y DEPARTAMENTOS
df_pos = df_pos.merge(df_dep, left_on='departamento', right_on='departamento', how='left')

# CREANDO DATAFRAME POSITIVOS POR SEXO Y ETAPA DE VIDA Y POR DIA
df_positivo = df_pos.groupby(['departamentocode', 'fecha_resultado', 'sexo', 'etapa_de_vida'])['departamentocode'].count().reset_index(name='cantidad')
df_positivo = df_positivo.rename(columns={'fecha_resultado': 'fecha'})

In [162]:
df_positivo.head()

,departamentocode,fecha,sexo,etapa_de_vida,cantidad
0,PE-AMA,2020-04-01,FEMENINO,ADOLESCENTE,3
1,PE-AMA,2020-04-01,FEMENINO,ADULTO,50
2,PE-AMA,2020-04-01,FEMENINO,JOVEN,14
3,PE-AMA,2020-04-01,FEMENINO,MAYOR,9
4,PE-AMA,2020-04-01,FEMENINO,NIÑO,6


In [ ]:
# -----------------------------------------------------------------
# JOIN DE POSITIVOS Y FALLECIDOS POR COVID
# -----------------------------------------------------------------
df_covid = df_positivo.merge(df_fallecido[['cantidad', 'departamentocode', 'fecha', 'sexo', 'etapa_de_vida']], on=['departamentocode', 'fecha', 'sexo', 'etapa_de_vida'], how='left')
df_covid = df_covid.rename(columns={'cantidad_x': 'positivo', 'cantidad_y': 'fallecido'})
df_covid['fallecido'] = df_covid['fallecido'].fillna(0)

In [163]:
df_covid.head()

,departamentocode,fecha,sexo,etapa_de_vida,positivo,fallecido,AnioMes
0,PE-AMA,2020-04-01,FEMENINO,ADOLESCENTE,2,0.0,2020-04
1,PE-AMA,2020-04-01,FEMENINO,ADULTO,48,0.0,2020-04
2,PE-AMA,2020-04-01,FEMENINO,JOVEN,14,0.0,2020-04
3,PE-AMA,2020-04-01,FEMENINO,MAYOR,6,1.0,2020-04
4,PE-AMA,2020-04-01,FEMENINO,NIÑO,4,0.0,2020-04


In [ ]:
# -----------------------------------------------------------------
# ETL MUNDIAL COVID
# -----------------------------------------------------------------

# CAMBIANDO EL CAMPO FECHA A TIPO STRING, FORMATO AAAA-MM-01
df_mun['date'] = df_mun['date'].astype('string')
df_mun['date'] = df_mun['date'].str[:7] + '-01'

In [164]:
df_mun.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [165]:

# RENOMBRANDO CAMPOS PARA FACILIDAD DE LECTURA
df_mun = df_mun.rename(columns={'iso_code': 'paisCode', 'location':'pais',  'continent':'continente', 'date': 'fecha'})

# AGRUPANDO PARA CREAR TABLE HECHOS COVID MUNDIAL
df_covid_mundial = df_mun.groupby(['paisCode', 'fecha'])['new_cases', 'new_deaths'].sum().reset_index()
df_covid_mundial = df_covid_mundial.rename(columns={'new_cases': 'positivo', 'new_deaths':'fallecido'})

# CREANDO DIMENSION PAIS
df_pais = df_mun.groupby(['paisCode', 'pais', 'continente'])['new_cases'].count().reset_index()
df_pais = df_pais[['paisCode', 'pais', 'continente']]


C:\Users\Miguel\AppData\Local\Temp\ipykernel_16612\3179240019.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_covid_mundial = df_mun.groupby(['paisCode', 'fecha'])['new_cases', 'new_deaths'].sum().reset_index()


In [166]:
df_covid_mundial.head()

,paisCode,fecha,positivo,fallecido
0,ABW,2020-01-03,0.0,0.0
1,ABW,2020-01-04,0.0,0.0
2,ABW,2020-01-05,0.0,0.0
3,ABW,2020-01-06,0.0,0.0
4,ABW,2020-01-07,0.0,0.0


In [167]:
# -----------------------------------------------------------------
# ETL CREANDO DIMENSION CALENDARIO
# -----------------------------------------------------------------

# HALLANDO FECHA MAXIMA
fec_max1 = df_covid['fecha'].max()
fec_max2 = df_covid_mundial['fecha'].max()
fec_max =  fec_max1 if fec_max1 > fec_max2 else fec_max2
fec_max = pd.to_datetime(fec_max).date()

# HALLANDO FECHA MINIMA
fec_min1 = df_covid['fecha'].min()
fec_min2 = df_covid_mundial['fecha'].min()
fec_min =  fec_min1 if fec_min1 > fec_min2 else fec_min2
fec_min = pd.to_datetime(fec_min).date()

mes_dict = {
    i + 1: mes
    for i, mes in enumerate(['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Setiembre', 'Octubre', 'Noviembre', 'Diciembre'])
}

df_calendario = pd.DataFrame({'fecha': pd.date_range(fec_min, fec_max)})
df_calendario['Año'] = df_calendario.fecha.dt.year
df_calendario['Trimestre'] = df_calendario.fecha.dt.quarter.map(lambda x :  'Trim. ' + str(x))
df_calendario['Mes'] = df_calendario.fecha.dt.month.map(lambda x: mes_dict[x])
df_calendario['Dia'] = df_calendario.fecha.dt.day
df_calendario['AñoMes'] = df_calendario.fecha.astype('string').str[:4] + df_calendario.fecha.astype('string').str[5:7]

In [168]:
df_calendario.head()

,fecha,Año,Trimestre,Mes,Dia,AñoMes
0,2020-03-01,2020,Trim. 1,Marzo,1,202003
1,2020-03-02,2020,Trim. 1,Marzo,2,202003
2,2020-03-03,2020,Trim. 1,Marzo,3,202003
3,2020-03-04,2020,Trim. 1,Marzo,4,202003
4,2020-03-05,2020,Trim. 1,Marzo,5,202003


In [169]:
# -----------------------------------------------------------------
# ETL CREANDO DIMENSION DEPARTAMENTO
# -----------------------------------------------------------------

# EN POWER BI, EN GRAFICO TIPO MAPAS, PARA QUE IDENTIFICA UN DEPARTAMENTO DE UN PAIS ES 
# NECESARIO CREAR UN CAMPO CON FORMATO >> DEPARTAMENTO (DEPARTAMENTO), (PAIS)
df_dep['departamentoPais'] = 'DEPARTAMENTO ' + df_dep['departamento'] + ', PERU'

In [170]:
df_dep.head()

,pais,paiscode,departamento,departamentocode,departamentoPais
0,PERU,PE,AMAZONAS,PE-AMA,"DEPARTAMENTO AMAZONAS, PERU"
1,PERU,PE,ANCASH,PE-ANC,"DEPARTAMENTO ANCASH, PERU"
2,PERU,PE,APURIMAC,PE-APU,"DEPARTAMENTO APURIMAC, PERU"
3,PERU,PE,AREQUIPA,PE-ARE,"DEPARTAMENTO AREQUIPA, PERU"
4,PERU,PE,AYACUCHO,PE-AYA,"DEPARTAMENTO AYACUCHO, PERU"


In [ ]:
df_covid.to_csv('covid.csv', index=False)
df_covid_mundial.to_csv('covid_mundial.csv', index=False)
df_pais.to_csv('dim_pais.csv', index=False)
df_dep.to_csv('dim_departamento.csv', index=False)
df_calendario.to_csv('dim_calendario.csv', index=False)